# Explaining output of the model


References:
- [SHAP](https://github.com/shap/shap#natural-language-example-transformers)
- [refactor: raise error during init if masker is None for PermutationExplainer](https://github.com/shap/shap/pull/3315)
- [BERT, explain yourself! ](https://colab.research.google.com/github/ml6team/quick-tips/blob/main/nlp/2021_04_22_shap_for_huggingface_transformers/explainable_transformers_using_shap.ipynb#scrollTo=das1RvNUrsE_)
- [Using SHAP (SHapley Additive exPlanations) to explain the predictions of a zero-shot transformer pipeline for text classification (using Huggingface)?](https://www.reddit.com/r/MLQuestions/comments/qc7uo7/using_shap_shapley_additive_explanations_to/)
- [Positive vs. Negative Sentiment Classification](https://shap.readthedocs.io/en/latest/example_notebooks/text_examples/sentiment_analysis/Positive%20vs.%20Negative%20Sentiment%20Classification.html)
- [text plot](https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/text.html)

In [1]:
# @title Environment running
running_local = False  # @param {type:"boolean"}
if running_local:
    running_colab = running_kaggle = False
else:
    running_colab = False  # @param {type:"boolean"}
    running_kaggle = True  # @param {type:"boolean"}

In [2]:
if running_colab:
    from google.colab import drive

    drive.mount("/content/drive")

In [3]:
if running_colab or running_kaggle:
    !pip install shap

## Loading the model

In [4]:
import numpy as np
import pandas as pd
import random
import shap
import torch
import torch.nn as nn

from transformers import BertTokenizer, BertModel

In [5]:
RANDOM_SEED = 103
MODEL_PATH = "neuralmind/bert-base-portuguese-cased"
TOKEN_MAX_LENGTH = 512

if running_local:
    GLASSDOOR_MODEL_PATH = "./bertimbau-glassdoor-reviews-epoch_5.bin"
    OVERSAMPLED_GLASSDOOR_MODEL_PATH = "./bertimbau-glassdoor-reviews-epoch_5.bin"
if running_colab:
    GLASSDOOR_MODEL_PATH = "/content/drive/MyDrive/UFMT/Gestão e Ciência de Dados/Disciplinas/14 - Seminário e Metodologia da Pesquisa/Projetos/glassdoor-reviews-analysis-nlp/train_model/bertimbau-glassdoor-reviews-epoch_5.bin"
    OVERSAMPLED_GLASSDOOR_MODEL_PATH = "/content/drive/MyDrive/UFMT/Gestão e Ciência de Dados/Disciplinas/14 - Seminário e Metodologia da Pesquisa/Projetos/glassdoor-reviews-analysis-nlp/train_model/bertimbau-glassdoor-reviews-oversampled-epoch_5.bin"
if running_kaggle:
    GLASSDOOR_MODEL_PATH = "/kaggle/input/bertimbau-glassdoor-reviews-epoch_5.bin/pytorch/bertimbau-glassdoor-reviews-epoch_5/1/bertimbau-glassdoor-reviews-epoch_5.bin"
    OVERSAMPLED_GLASSDOOR_MODEL_PATH = "/kaggle/input/bertimbau-glassdoor-reviews-oversammpled_5.bin/pytorch/bertimbau-glassdoor-reviews-oversampled-epoch_5/1/bertimbau-glassdoor-reviews-oversampled-epoch_5.bin"

In [6]:
torch.manual_seed(RANDOM_SEED)

In [7]:
random.seed(RANDOM_SEED)

In [8]:
np.random.seed(RANDOM_SEED)

In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"There are {torch.cuda.device_count()} GPU(s) available.")
    print("Device name:", torch.cuda.get_device_name(0))
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


In [10]:
if running_colab:
    dataset = pd.read_csv(
        "/content/drive/MyDrive/UFMT/Gestão e Ciência de Dados/Disciplinas/14 - Seminário e Metodologia da Pesquisa/Projetos/glassdoor-reviews-analysis-nlp/data_preparation/glassdoor_reviews_annotated.csv"
    )
else:
    if running_kaggle:
        dataset = pd.read_csv(
            "/kaggle/input/glassdoor-reviews-annotated/glassdoor_reviews_annotated.csv"
        )
    else:
        dataset = pd.read_csv("../data_preparation/glassdoor_reviews_annotated.csv")

In [11]:
filtered_dataset = dataset.filter(["review_text", "sentiment"])

In [12]:
filtered_dataset.shape

(2566, 2)

In [13]:
filtered_dataset["sentiment"].value_counts()

sentiment
 1    1284
-1    1035
 0     247
Name: count, dtype: int64

In [14]:
filtered_dataset.head(2)

,review_text,sentiment
0,"Companheirismo entre os colegas, oportunidade ...",1
1,Não tive nenhum ponto negativo,0


Replace negative sentiment (-1) to 2, to avoid PyTorch errors.

In [15]:
filtered_dataset["sentiment"] = filtered_dataset["sentiment"].apply(
    lambda x: 2 if x == -1 else x
)

In [16]:
filtered_dataset["sentiment"].value_counts()

sentiment
1    1284
2    1035
0     247
Name: count, dtype: int64

In [17]:
num_labels = len(filtered_dataset["sentiment"].value_counts())

In [18]:
num_labels

3

In [19]:
class_names = ["neutral", "positive", "negative"]

## Creating a PyTorch Model

In [20]:
class GlassdoorReviewsClassifier(nn.Module):
    def __init__(self, num_labels):
        super(GlassdoorReviewsClassifier, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_PATH)
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 300),
            nn.ReLU(),
            nn.Linear(300, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, num_labels),
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = outputs["last_hidden_state"][:, 0, :]
        x = self.classifier(x)
        return x

In [21]:
model = GlassdoorReviewsClassifier(num_labels).to(device)
model.load_state_dict(torch.load(GLASSDOOR_MODEL_PATH, map_location=device))
model.eval()

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()


GlassdoorReviewsClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

## Prediction Explaining

In [22]:
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [23]:
def convert_to_str(input_value):
    if isinstance(input_value, np.ndarray):
        input_str = " ".join(input_value)
    else:
        input_str = input_value

    return input_str

In [24]:
def predict_sentiment(texts):
    outputs = []
    for txt in texts:
        encoded_texts = tokenizer(
            convert_to_str(texts),
            max_length=TOKEN_MAX_LENGTH,
            add_special_tokens=True,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        input_ids = encoded_texts["input_ids"].to(device)
        attention_mask = encoded_texts["attention_mask"].to(device)

        with torch.no_grad():
            output = model(input_ids, attention_mask)
            probabilities = torch.nn.functional.softmax(output, dim=1)
            outputs.append(probabilities.cpu().numpy())

    return np.concatenate(outputs, axis=0)

In [25]:
def explain_prediction(text):
    output_probabilities = predict_sentiment(text)
    predicted_index = np.argmax(output_probabilities)
    predicted_class = class_names[predicted_index]

    print(f"Review: {text[0]}")
    print(f"Predicted sentiment: {predicted_class}")

    explainer = shap.Explainer(
        model=predict_sentiment, masker=tokenizer, output_names=class_names
    )
    shap_values = explainer(text)

    # print(shap_values)

    shap.plots.text(shap_values[0, :, predicted_class])

### Explaining Neutral Reviews

In [26]:
neutral_sample_df = filtered_dataset[filtered_dataset["sentiment"] == 0].sample(
    n=3, random_state=RANDOM_SEED
)

In [27]:
explain_prediction([neutral_sample_df.iloc[0]["review_text"]])

Review: não nada a citar sobre
Predicted sentiment: neutral


2024-04-16 04:10:49.084520: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 04:10:49.084626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 04:10:49.222988: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [28]:
explain_prediction([neutral_sample_df.iloc[1]["review_text"]])

Review: Considero que os problemas que encontrei foram prontamente resolvidos.
Predicted sentiment: neutral


  0%|          | 0/272 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:11, 11.49s/it]               


In [29]:
explain_prediction([neutral_sample_df.iloc[2]["review_text"]])

Review: Bom até o momento eu não identifiquei pontos negativos na empresa. Eu acho que a burocracia para retirada de material mas eu entendo perfeitamente que tem que se ter um controle
Predicted sentiment: neutral


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:13, 13.58s/it]               


### Explaining Positive Reviews

In [30]:
positive_sample_df = filtered_dataset[filtered_dataset["sentiment"] == 1].sample(
    n=3, random_state=42
)

In [31]:
explain_prediction([positive_sample_df.iloc[0]["review_text"]])

Review: Empresa que tem uma grande carteira de clientes, ótima para aprendizado. Bem localizado em Cuiabá. Ótimo pra quem está querendo começar carreira.
Predicted sentiment: positive


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:12, 12.94s/it]               


In [32]:
explain_prediction([positive_sample_df.iloc[1]["review_text"]])

Review: Oferece plano de saúde participativo, é um diferencial pois poucas empresas oferecem isso na cidade. Oferece refeição no local. Horário reduzido tbm é bom
Predicted sentiment: positive


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:13, 13.31s/it]               


In [33]:
explain_prediction([positive_sample_df.iloc[2]["review_text"]])

Review: Empresa boa de se trabalhar, pessoas competentes, alimentação de excelente qualidade, terra muito boa e produtiva, local não tão afastado da cidade.
Predicted sentiment: positive


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:12, 12.97s/it]               


### Explaining Negative Reviews

In [34]:
negative_sample_df = filtered_dataset[filtered_dataset["sentiment"] == 2].sample(
    n=3, random_state=42
)

In [35]:
explain_prediction([negative_sample_df.iloc[0]["review_text"]])

Review: Poderia estar melhor desenvolvimento o programa para não gestores. Poderia ter outras formas de reconhecimento dos colaboradores. Os gestores poderiam olhar com mais atenção para seleções internas.
Predicted sentiment: negative


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:13, 13.55s/it]               


In [36]:
explain_prediction([negative_sample_df.iloc[1]["review_text"]])

Review: Salário e benefícios a baixo do mercado
Predicted sentiment: negative


In [37]:
explain_prediction([negative_sample_df.iloc[2]["review_text"]])

Review: Alguns atrasos de pagamentos, porém provenientes de crises no setor público.
Predicted sentiment: negative


  0%|          | 0/240 [00:00<?, ?it/s]

## Explaining oversampled model

In [38]:
model = GlassdoorReviewsClassifier(num_labels).to(device)
model.load_state_dict(torch.load(OVERSAMPLED_GLASSDOOR_MODEL_PATH, map_location=device))
# model.eval()

<All keys matched successfully>

### Explaining Neutral Reviews

In [39]:
explain_prediction([neutral_sample_df.iloc[0]["review_text"]])

Review: não nada a citar sobre
Predicted sentiment: neutral


In [40]:
explain_prediction([neutral_sample_df.iloc[1]["review_text"]])

Review: Considero que os problemas que encontrei foram prontamente resolvidos.
Predicted sentiment: neutral


  0%|          | 0/272 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:11, 11.49s/it]               


In [41]:
explain_prediction([neutral_sample_df.iloc[2]["review_text"]])

Review: Bom até o momento eu não identifiquei pontos negativos na empresa. Eu acho que a burocracia para retirada de material mas eu entendo perfeitamente que tem que se ter um controle
Predicted sentiment: neutral


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:13, 13.57s/it]               


### Explaining Positive Reviews

In [42]:
explain_prediction([positive_sample_df.iloc[0]["review_text"]])

Review: Empresa que tem uma grande carteira de clientes, ótima para aprendizado. Bem localizado em Cuiabá. Ótimo pra quem está querendo começar carreira.
Predicted sentiment: positive


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:12, 12.93s/it]               


In [43]:
explain_prediction([positive_sample_df.iloc[1]["review_text"]])

Review: Oferece plano de saúde participativo, é um diferencial pois poucas empresas oferecem isso na cidade. Oferece refeição no local. Horário reduzido tbm é bom
Predicted sentiment: positive


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:13, 13.27s/it]               


In [44]:
explain_prediction([positive_sample_df.iloc[2]["review_text"]])

Review: Empresa boa de se trabalhar, pessoas competentes, alimentação de excelente qualidade, terra muito boa e produtiva, local não tão afastado da cidade.
Predicted sentiment: positive


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:12, 13.00s/it]               


### Explaining Negative Reviews

In [45]:
explain_prediction([negative_sample_df.iloc[0]["review_text"]])

Review: Poderia estar melhor desenvolvimento o programa para não gestores. Poderia ter outras formas de reconhecimento dos colaboradores. Os gestores poderiam olhar com mais atenção para seleções internas.
Predicted sentiment: negative


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:13, 13.53s/it]               


In [46]:
explain_prediction([negative_sample_df.iloc[1]["review_text"]])

Review: Salário e benefícios a baixo do mercado
Predicted sentiment: negative


In [47]:
explain_prediction([negative_sample_df.iloc[2]["review_text"]])

Review: Alguns atrasos de pagamentos, porém provenientes de crises no setor público.
Predicted sentiment: negative


  0%|          | 0/240 [00:00<?, ?it/s]